# Weather Features (Optional)


Optional: download and merge weather data into features for improved forecasting.

In [1]:
# Environment setup
import sys, subprocess
from pathlib import Path

print('Python:', sys.executable)
repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
subprocess.run(['pip', 'install', '-e', str(repo_root)], check=True)

Python: /Users/pratik_n/Downloads/gridpulse/.venv/bin/python
Obtaining file:///Users/pratik_n/Downloads/gridpulse
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for gridpulse (pyproject.toml): started
  Building editable for gridpulse (pyproject.toml): finished with status 'done'
  Created wheel for gridpulse: filename=gridpulse-0.1.0-0.editable-py3-none-any.whl size=6066 sha256=de0ccda0a7cd3f1b9742ec129a539afbbefc26ea4a1c441acf5d2c5a40261de0
  Stored in directory: /private/var/folders/pk/ksqc41pj60q3589gfwhqz5tw0000gn/T/pip-

CompletedProcess(args=['pip', 'install', '-e', '/Users/pratik_n/Downloads/gridpulse'], returncode=0)

In [3]:
import subprocess
from pathlib import Path
import shutil

# ensure raw CSV exists
raw_csv = repo_root / "data" / "raw" / "time_series_60min_singleindex.csv"
if not raw_csv.exists():
    nested = next((repo_root / "data" / "raw").glob("opsd-time_series-*/time_series_60min_singleindex.csv"), None)
    if nested:
        shutil.copy(nested, raw_csv)

subprocess.run([
    "python", "-m", "gridpulse.data_pipeline.download_weather",
    "--out", "data/raw", "--start", "2017-01-01", "--end", "2020-12-31"
], check=True, cwd=str(repo_root))

subprocess.run([
    "python", "-m", "gridpulse.data_pipeline.build_features",
    "--in", "data/raw", "--out", "data/processed",
    "--weather", "data/raw/weather_berlin_hourly.csv"
], check=True, cwd=str(repo_root))


/Users/pratik_n/Downloads/gridpulse/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Fetching 2017-01-01 to 2017-12-31 ...
Fetching 2018-01-01 to 2018-12-31 ...
Fetching 2019-01-01 to 2019-12-31 ...
Fetching 2020-01-01 to 2020-12-30 ...
Fetching 2020-12-31 to 2020-12-31 ...
Saved: data/raw/weather_berlin_hourly.csv
Saved: data/processed/features.parquet
Rows: 32856 | Columns: 44


CompletedProcess(args=['python', '-m', 'gridpulse.data_pipeline.build_features', '--in', 'data/raw', '--out', 'data/processed', '--weather', 'data/raw/weather_berlin_hourly.csv'], returncode=0)